### OpenCV processed Trump Footage

In [1]:
import numpy as np
import cv2, math, dlib, random
from itertools import product

cap = cv2.VideoCapture('./BBC_Trump.mp4')

FPS = cap.get(cv2.CAP_PROP_FPS)               # Frame Per Second
F_Count = cap.get(cv2.CAP_PROP_FRAME_COUNT)   # frame count
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))    # frame width
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))   # frame height

seg_count = F_Count/12                        # seg = 12 segmentation
print(f'FPS : {FPS:.2f} f/s,\tF_Count : {F_Count},\tw : {w},\th :　{h}')

fourcc = cv2.VideoWriter_fourcc(*'XVID')      # 使用 XVID 編碼
out = cv2.VideoWriter('./TrumpFootage.mp4', fourcc, 30, (1280, 720))     # save video file
pos=(10,30);    font=0;    color=(255,255,255)

r = math.ceil((seg_count)/90)

bs = cv2.createBackgroundSubtractorMOG2()  # create object
sift = cv2.xfeatures2d.SIFT_create()    

while cap.isOpened():
    ret, frame = cap.read()
    if not ret  : break
    C_Count = cap.get(cv2.CAP_PROP_POS_FRAMES)

    ##### 1. Original
    if 0 < C_Count < seg_count*1:
        cv2.putText(frame, f'1. Original', (1100, 700), font, 1, color, 2)
            
    ##### 2. BGR2RBG
    elif seg_count*1 <= C_Count < seg_count*2:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        cv2.putText(frame, f'2. BGR2RBG', (1070, 700), font, 1, color, 2)
    
    ##### 3. BGR2HSV
    elif  seg_count*2 <= C_Count < seg_count*3:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        cv2.putText(frame, f'3. BGR2HSV', (1070, 700), font, 1, color, 2)

    ##### 4. GaussianBlur or Sharpen
    elif  seg_count*3 <= C_Count < seg_count*4:
        ### GaussianBlur
        # frame = cv2.GaussianBlur(frame, (9, 9), 0)
        # cv2.putText(frame, f'4. GaussianBlur', (1020, 700), font, 1, color, 2)
        
        ### Sharpen
        kernel_sharp1 = np.array([[-1,-1,-1],
                                  [-1,9,-1],
                                  [-1,-1,-1]])
        kernel_sharp2 = np.array([[1,1,1],
                                  [1,-7,1],
                                  [1,1,1]])
        out1 = cv2.filter2D(frame, cv2.CV_64F, kernel_sharp1)
        frame = cv2.convertScaleAbs(out1)
        cv2.putText(frame, f'4. Sharpen by CNN with self-defined kernel', (560, 700), font, 1, color, 2)
        
            
    ##### 5. Flip and addWeighted
    elif  seg_count*4 <= C_Count < seg_count*5:
        frame1 = cv2.flip(frame, 1)
        frame = cv2.addWeighted(frame, 0.5, frame1, 0.5, 0)
        cv2.putText(frame, f'5. Flip and addWeighted', (890, 700), font, 1, color, 2)
        
    ##### 6. Sobel and 7. Canny
    elif  seg_count*5 <= C_Count < seg_count*6:
        sobelx = cv2.Sobel(frame[:,:w//2], cv2.CV_64F, 1, 0, ksize=-1)
        sobely = cv2.Sobel(frame[:,:w//2], cv2.CV_64F, 0, 1, ksize=-1)
        sobelx = cv2.convertScaleAbs(sobelx)          
        sobely = cv2.convertScaleAbs(sobely)
        frame1 =  cv2.addWeighted(sobelx, 0.5, sobely, 0.5, 0)  

        frame2 = cv2.cvtColor(cv2.Canny(frame[:,w//2:w], 64, 192), cv2.COLOR_GRAY2BGR)
        
        frame=np.hstack([frame1, frame2])
        
        cv2.putText(frame, f'6. Sobel', (20, 60), font, 1, color, 2)
        cv2.putText(frame, f'7. Canny', (650, 60), font, 1, color, 2)
        cv2.putText(frame, f'6. Sobel', (1120, 650), font, 1, color, 2)
        cv2.putText(frame, f'7. Canny', (1120, 700), font, 1, color, 2)
        
    ##### 7. Canny and 8. Laplacian
    elif  seg_count*6 <= C_Count < seg_count*7:
        frame1 = cv2.cvtColor(cv2.Canny(frame[:,:w//2], 64, 192), cv2.COLOR_GRAY2BGR)
        
        lapl = cv2.Laplacian(frame[:,w//2:], cv2.CV_64F)
        frame2 = cv2.convertScaleAbs(lapl)

        frame=np.hstack([frame1, frame2])
        
        cv2.putText(frame, f'7. Canny', (20, 60), font, 1, color, 2)
        cv2.putText(frame, f'8. Laplacian', (650, 60), font, 1, color, 2)
        cv2.putText(frame, f'7. Canny', (1070, 650), font, 1, color, 2)
        cv2.putText(frame, f'8. Laplacian', (1070, 700), font, 1, color, 2)
        
    ##### 9. Rotation
    elif  seg_count*7 <= C_Count < seg_count*8:
        ang=(C_Count%seg_count)/seg_count*360*r
        M1 = cv2.getRotationMatrix2D((w/2, h/2), ang, (C_Count%seg_count)/(seg_count))
        frame = cv2.warpAffine(frame, M1, (w, h))
        cv2.putText(frame, f'9. Rotation', (1070, 700), font, 1, color, 2)
    
    ##### 10. Draw contours and rectangle by MOG2
    elif seg_count*8 <= C_Count < seg_count*9:
        gray = bs.apply(frame)
        
        mask = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)[1]
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=10)
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for c in contours:
            if cv2.contourArea(c) < 15000:
                continue

            cv2.drawContours(frame, contours, -1, (0, 255, 255), 2)   
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 2)
        
        cv2.putText(frame, f'10. Draw contours and rectangle by MOG2', (580, 700), font, 1, color, 2)
            
    ##### 11. Gradient
    elif seg_count*9 <= C_Count < seg_count*10:
                
        kernel = np.ones((3, 3), np.uint8)  
        frame = cv2.morphologyEx(frame, cv2.MORPH_GRADIENT, kernel)
        cv2.putText(frame, f'11. Gradient', (1070, 700), font, 1, color, 2)


    #### 12. Sift
    else:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        kps = sift.detect(frame, None)
        frame= cv2.drawKeypoints(frame, kps, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)     
        cv2.putText(frame, f'12. SIFT', (1120, 700), font, 1, color, 2)
    
    
    cv2.putText(frame, f'{C_Count:.0f}/{F_Count:.0f} frames, FPS : {FPS:.0f}', (10, 700), font, 1, color, 2)
    cv2.imshow('Trump_footage', frame)
    out.write(frame)
    
    if cv2.waitKey(1) == 27:          # c==27 (ascii code) key escape
        break
        
cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

FPS : 29.97 f/s,	F_Count : 2251.0,	w : 1280,	h :　720


-1